In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import os

import arrow
import pandas as pd

Everything is for Spain (ES)

In [7]:
# Installed Capacity
capacity_df = pd.read_csv("Installed Capacity per Production Type_202201010000-202301010000.csv")

capacity_df.loc[:, "Balancing Region"] = "ES"
capacity_df.loc[:, "datetime"] = "2022-01-01"
capacity_df = capacity_df.rename(columns={"Production Type": "Fuel Type", "2022 [MW]": "Capacity (MW)"})

capacity_df.to_csv("installed_capacity.csv", index=False)

In [17]:
# Generation
generation_df = pd.read_csv("Actual Generation per Production Type_202211010000-202211020000.csv")

generation_df.loc[:, "Balancing Region"] = "ES"
generation_df.loc[:, "datetime"] = generation_df.loc[:, "MTU"].apply(lambda x: arrow.get(x.split(" - ")[0], "DD.MM.YYYY HH:mm").format("YYYY-MM-DD HH:mm"))

generation_df = generation_df.drop(columns=["MTU", "Area"])
generation_df = generation_df.rename(columns={c: c.split("  - Actual Aggregated [MW]")[0] + " (MW)" for c in generation_df.columns if "Actual Aggregated [MW]" in c})

generation_df.to_csv("generation.csv", index=False)

In [22]:
# Imports and Exports
imports_exports_df = pd.read_csv("Cross-Border Physical Flow_202211010000-202211020000.csv")

imports_exports_df.loc[:, "Balancing Region"] = "ES"
imports_exports_df.loc[:, "datetime"] = imports_exports_df.loc[:, "Time (UTC)"].apply(lambda x: arrow.get(x.split(" - ")[0], "DD.MM.YYYY HH:mm").format("YYYY-MM-DD HH:mm"))

imports_exports_df = imports_exports_df.rename(columns={"BZN|FR > BZN|ES [MW]": "FR import (MW)", "BZN|ES > BZN|FR [MW]": "FR export (MW)"})
imports_exports_df = imports_exports_df.drop(columns=["Time (UTC)"])

imports_exports_df.to_csv("imports_exports.csv", index=False)

In [26]:
# Load forecast
load_forecast_df = pd.read_csv("Total Load - Day Ahead _ Actual_202211010000-202211020000.csv")

load_forecast_df.loc[:, "datetime"] = load_forecast_df.loc[:, "Time (UTC)"].apply(lambda x: arrow.get(x.split(" - ")[0], "DD.MM.YYYY HH:mm").format("YYYY-MM-DD HH:mm"))
load_forecast_df = load_forecast_df.drop(columns=["Time (UTC)"])
load_forecast_df = load_forecast_df.rename(columns={"Day-ahead Total Load Forecast [MW] - BZN|ES": "Demand Forecast (MW)"})

load_forecast_df.to_csv("load_forecast.csv", index=False)